In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Define the list of international indices and equities
indices = ['^GSPC', '^IXIC', '^DJI', '^FTSE', '^N225']
equities = ['AAPL', 'GOOGL', 'AMZN', 'FB', 'TSLA']

# Define the date range
start_date = '2010-01-01'
end_date = '2023-05-01'

# Fetch the data
indices_data = yf.download(indices, start=start_date, end=end_date)['Adj Close']
equities_data = yf.download(equities, start=start_date, end=end_date)['Adj Close']

# Compute daily returns
indices_returns = indices_data.pct_change().dropna()
equities_returns = equities_data.pct_change().dropna()

# Calculate cumulative returns
indices_cumulative_returns = (1 + indices_returns).cumprod()
equities_cumulative_returns = (1 + equities_returns).cumprod()

# Calculate maximum drawdown
def calculate_max_drawdown(returns):
    cumulative_returns = (1 + returns).cumprod()
    previous_peaks = cumulative_returns.cummax()
    drawdowns = (cumulative_returns - previous_peaks) / previous_peaks
    max_drawdown = drawdowns.min()
    return max_drawdown

indices_max_drawdowns = indices_returns.groupby(indices_returns.index.year).apply(calculate_max_drawdown)
equities_max_drawdowns = equities_returns.groupby(equities_returns.index.year).apply(calculate_max_drawdown)

# Calculate Sharpe ratio
risk_free_rate = 0.02  # Assuming a risk-free rate of 2%
indices_sharpe_ratio = (indices_returns.mean() - risk_free_rate) / indices_returns.std()
equities_sharpe_ratio = (equities_returns.mean() - risk_free_rate) / equities_returns.std()

# Calculate Sortino ratio
downside_threshold = 0  # Assuming downside threshold of 0 (negative returns)
indices_downside_returns = indices_returns[indices_returns < downside_threshold]
equities_downside_returns = equities_returns[equities_returns < downside_threshold]

indices_sortino_ratio = (indices_returns.mean() - risk_free_rate) / indices_downside_returns.std()
equities_sortino_ratio = (equities_returns.mean() - risk_free_rate) / equities_downside_returns.std()

# Display the results
print("Index Daily Returns:")
print(indices_returns.head())
print("\nIndex Cumulative Returns:")
print(indices_cumulative_returns.tail())
print("\nIndex Max Drawdowns:")
print(indices_max_drawdowns)
print("\nIndex Sharpe Ratios:")
print(indices_sharpe_ratio)
print("\nIndex Sortino Ratios:")
print(indices_sortino_ratio)

print("\nEquity Daily Returns:")
print(equities_returns.head())
print("\nEquity Cumulative Returns:")
print(equities_cumulative_returns.tail())
print("\nEquity Max Drawdowns:")
print(equities_max_drawdowns)
print("\nEquity Sharpe Ratios:")
print(equities_sharpe_ratio)
print("\nEquity Sortino Ratios:")
print(equities_sortino_ratio)

# Plot cumulative returns
plt.figure(figsize=(10, 6))
for index in indices_cumulative_returns.columns:
    plt.plot(indices_cumulative_returns.index, indices_cumulative_returns[index], label=index)
plt.title('Index Cumulative Returns')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
for equity in equities_cumulative_returns.columns:
    plt.plot(equities_cumulative_returns.index, equities_cumulative_returns[equity], label=equity)
plt.title('Equity Cumulative Returns')
plt.legend()
plt.show()
